<a href="https://colab.research.google.com/github/mr-alamdari/NLP-Text-Generation/blob/main/NLP_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import numpy as np
from pickle import dump, load

In [2]:
def read_file(path):
  with open(path) as f:
    my_str = f.read()
  return my_str

In [3]:
!wget https://raw.githubusercontent.com/mr-alamdari/NLP-Text-Generation/main/moby_dick_four_chapters.txt

--2022-05-03 11:42:41--  https://raw.githubusercontent.com/mr-alamdari/NLP-Text-Generation/main/moby_dick_four_chapters.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62685 (61K) [text/plain]
Saving to: ‘moby_dick_four_chapters.txt’

moby_dick_four_chap 100%[===================>]  61.22K  --.-KB/s    in 0.001s  

2022-05-03 11:42:41 (66.8 MB/s) - ‘moby_dick_four_chapters.txt’ saved [62685/62685]



In [4]:
mobydic = read_file('moby_dick_four_chapters.txt')

In [5]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [6]:
nlp.max_length = 1198623

In [7]:
seperate_punc = lambda doc: [token.text.lower() for token in nlp(doc) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [8]:
tokens = seperate_punc(mobydic)

In [10]:
tokens[20:40]

['nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore',
 'i',
 'thought',
 'i',
 'would',
 'sail',
 'about',
 'a',
 'little',
 'and',
 'see',
 'the',
 'watery',
 'part']

In [11]:
len(tokens)

11338

In [12]:
train_len = 25 + 1 # 25 training words , then one target word

text_sequences = [tokens[i-train_len: i] for i in range(train_len, len(tokens))]

In [13]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [14]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [15]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [16]:
' '.join(text_sequences[3])

'some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i thought'

In [17]:
import tensorflow as tf

In [18]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [19]:
tokenizer.fit_on_texts(text_sequences)

In [20]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [21]:
sequences = np.array(sequences)

In [22]:
print(sequences[0])

[ 956   14  263   51  261  408   87  219  129  111  954  260   50   43
   38  315    7   23  546    3  150  259    6 2712   14   24]


In [23]:
print(sequences[1])

[  14  263   51  261  408   87  219  129  111  954  260   50   43   38
  315    7   23  546    3  150  259    6 2712   14   24  957]


In [24]:
print(sequences[2])

[ 263   51  261  408   87  219  129  111  954  260   50   43   38  315
    7   23  546    3  150  259    6 2712   14   24  957    5]


In [25]:
print(sequences[3])

[  51  261  408   87  219  129  111  954  260   50   43   38  315    7
   23  546    3  150  259    6 2712   14   24  957    5   60]


In [26]:
for i in sequences[0]:
  print(i, tokenizer.index_word[i])

956 call
14 me
263 ishmael
51 some
261 years
408 ago
87 never
219 mind
129 how
111 long
954 precisely
260 having
50 little
43 or
38 no
315 money
7 in
23 my
546 purse
3 and
150 nothing
259 particular
6 to
2712 interest
14 me
24 on


In [27]:
for i, count in enumerate(sorted(tokenizer.word_counts.items(), key=lambda x: -x[1])):
  if i == 30:
    break
  print(i, count)

0 ('the', 15540)
1 ('a', 10377)
2 ('and', 9646)
3 ('of', 8287)
4 ('i', 7150)
5 ('to', 6497)
6 ('in', 5647)
7 ('it', 4238)
8 ('that', 3770)
9 ('he', 3247)
10 ('his', 3139)
11 ('was', 2886)
12 ('but', 2652)
13 ('me', 2471)
14 ('with', 2392)
15 ('as', 2366)
16 ('at', 2184)
17 ('this', 2158)
18 ('you', 2158)
19 ('is', 1950)
20 ('all', 1872)
21 ('for', 1820)
22 ('my', 1786)
23 ('on', 1716)
24 ('be', 1716)
25 ("'s", 1691)
26 ('not', 1534)
27 ('from', 1508)
28 ('there', 1456)
29 ('one', 1300)


In [28]:
vocab_size=len(tokenizer.word_counts)

In [29]:
vocab_size

2717

In [30]:
x = sequences[:, :-1]

In [31]:
y = sequences[:, -1]

In [32]:
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size+1)

In [33]:
seq_len = x.shape[1]

In [34]:
def create_model(vocab_size, seq_len):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, seq_len, input_length=seq_len))
  model.add(tf.keras.layers.LSTM(seq_len, return_sequences=True))
  model.add(tf.keras.layers.LSTM(seq_len))
  model.add(tf.keras.layers.Dense(seq_len, activation='relu'))
  model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

  model.summary()
  return model

In [35]:
model = create_model(vocab_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67950     
                                                                 
 lstm (LSTM)                 (None, 25, 25)            5100      
                                                                 
 lstm_1 (LSTM)               (None, 25)                5100      
                                                                 
 dense (Dense)               (None, 25)                650       
                                                                 
 dense_1 (Dense)             (None, 2718)              70668     
                                                                 
Total params: 149,468
Trainable params: 149,468
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(x, y, batch_size=32, epochs=200, verbose=2)

Epoch 1/200
354/354 - 8s - loss: 6.1576 - accuracy: 0.0517 - 8s/epoch - 23ms/step
Epoch 2/200
354/354 - 8s - loss: 6.0735 - accuracy: 0.0530 - 8s/epoch - 23ms/step
Epoch 3/200
354/354 - 8s - loss: 5.9708 - accuracy: 0.0541 - 8s/epoch - 23ms/step
Epoch 4/200
354/354 - 8s - loss: 5.8935 - accuracy: 0.0595 - 8s/epoch - 23ms/step
Epoch 5/200
354/354 - 8s - loss: 5.8123 - accuracy: 0.0656 - 8s/epoch - 23ms/step
Epoch 6/200
354/354 - 8s - loss: 5.7355 - accuracy: 0.0663 - 8s/epoch - 24ms/step
Epoch 7/200
354/354 - 8s - loss: 5.6799 - accuracy: 0.0681 - 8s/epoch - 23ms/step
Epoch 8/200
354/354 - 8s - loss: 5.6341 - accuracy: 0.0667 - 8s/epoch - 23ms/step
Epoch 9/200
354/354 - 8s - loss: 5.5914 - accuracy: 0.0674 - 8s/epoch - 23ms/step
Epoch 10/200
354/354 - 8s - loss: 5.5460 - accuracy: 0.0682 - 8s/epoch - 23ms/step
Epoch 11/200
354/354 - 8s - loss: 5.5051 - accuracy: 0.0703 - 8s/epoch - 23ms/step
Epoch 12/200
354/354 - 8s - loss: 5.4649 - accuracy: 0.0714 - 8s/epoch - 23ms/step
Epoch 13/200


In [39]:
model.save('mobydic_model.h5')

In [40]:
dump(tokenizer, open('simpleTokenizer', 'wb'))

In [41]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
  output_text = []
  input_text = seed_text
  for i in range(num_gen_words):
    encoded_text = tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded = tf.keras.preprocessing.sequence.pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
    pred_word = model.predict(pad_encoded, verbose=0)[0]
    pred_word_ind = np.argmax(pred_word)
    pred_word = tokenizer.index_word[pred_word_ind]
    input_text += ' '+pred_word
    output_text.append(pred_word)
  return ' '.join(output_text)

In [42]:
num_gen_words = 20
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

'aggregated years stand war than ye this harpooneer might be and exactly try the piece of knowing boots of him'

In [43]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 8s - loss: 1.3031 - accuracy: 0.6761 - 8s/epoch - 23ms/step
Epoch 2/100
354/354 - 8s - loss: 1.3128 - accuracy: 0.6713 - 8s/epoch - 23ms/step
Epoch 3/100
354/354 - 8s - loss: 1.2964 - accuracy: 0.6795 - 8s/epoch - 23ms/step
Epoch 4/100
354/354 - 8s - loss: 1.2651 - accuracy: 0.6858 - 8s/epoch - 23ms/step
Epoch 5/100
354/354 - 8s - loss: 1.2750 - accuracy: 0.6818 - 8s/epoch - 23ms/step
Epoch 6/100
354/354 - 8s - loss: 1.2462 - accuracy: 0.6910 - 8s/epoch - 23ms/step
Epoch 7/100
354/354 - 8s - loss: 1.2508 - accuracy: 0.6844 - 8s/epoch - 23ms/step
Epoch 8/100
354/354 - 8s - loss: 1.2353 - accuracy: 0.6917 - 8s/epoch - 23ms/step
Epoch 9/100
354/354 - 8s - loss: 1.2318 - accuracy: 0.6932 - 8s/epoch - 23ms/step
Epoch 10/100
354/354 - 8s - loss: 1.2352 - accuracy: 0.6933 - 8s/epoch - 23ms/step
Epoch 11/100
354/354 - 8s - loss: 1.2122 - accuracy: 0.7016 - 8s/epoch - 23ms/step
Epoch 12/100
354/354 - 8s - loss: 1.2036 - accuracy: 0.6993 - 8s/epoch - 23ms/step
Epoch 13/100


In [44]:
num_gen_words = 40
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

'i never unaccountable hours and had not help the decks in anxious sort of fooling it from what i told it for he continued the captain and to the answer i am two and nine at my bit of short'

In [45]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 8s - loss: 0.7558 - accuracy: 0.8128 - 8s/epoch - 23ms/step
Epoch 2/100
354/354 - 8s - loss: 0.7329 - accuracy: 0.8169 - 8s/epoch - 23ms/step
Epoch 3/100
354/354 - 8s - loss: 0.7533 - accuracy: 0.8122 - 8s/epoch - 23ms/step
Epoch 4/100
354/354 - 8s - loss: 0.7293 - accuracy: 0.8211 - 8s/epoch - 24ms/step
Epoch 5/100
354/354 - 8s - loss: 0.7326 - accuracy: 0.8192 - 8s/epoch - 23ms/step
Epoch 6/100
354/354 - 8s - loss: 0.7416 - accuracy: 0.8119 - 8s/epoch - 23ms/step
Epoch 7/100
354/354 - 8s - loss: 0.7505 - accuracy: 0.8120 - 8s/epoch - 23ms/step
Epoch 8/100
354/354 - 8s - loss: 0.7325 - accuracy: 0.8221 - 8s/epoch - 23ms/step
Epoch 9/100
354/354 - 8s - loss: 0.7131 - accuracy: 0.8230 - 8s/epoch - 23ms/step
Epoch 10/100
354/354 - 8s - loss: 0.7331 - accuracy: 0.8134 - 8s/epoch - 23ms/step
Epoch 11/100
354/354 - 8s - loss: 0.7835 - accuracy: 0.8002 - 8s/epoch - 23ms/step
Epoch 12/100
354/354 - 8s - loss: 0.7418 - accuracy: 0.8075 - 8s/epoch - 23ms/step
Epoch 13/100


In [46]:
model.save('mobydic_model.h5')
num_gen_words = 40
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

"of spending a sufferable night unless in some other person 's looking but the bench i think the piece of life for hands glided out to those place in him and if it 's turned over upon a grunt and"

In [ ]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 9s - loss: 0.7334 - accuracy: 0.8170 - 9s/epoch - 26ms/step
Epoch 2/100
354/354 - 9s - loss: 0.7685 - accuracy: 0.8062 - 9s/epoch - 26ms/step
Epoch 3/100
354/354 - 9s - loss: 0.7148 - accuracy: 0.8165 - 9s/epoch - 25ms/step
Epoch 4/100
354/354 - 9s - loss: 0.6765 - accuracy: 0.8288 - 9s/epoch - 25ms/step
Epoch 5/100
354/354 - 9s - loss: 0.6535 - accuracy: 0.8378 - 9s/epoch - 25ms/step
Epoch 6/100
354/354 - 9s - loss: 0.6855 - accuracy: 0.8276 - 9s/epoch - 25ms/step
Epoch 7/100
354/354 - 9s - loss: 0.6966 - accuracy: 0.8198 - 9s/epoch - 25ms/step
Epoch 8/100
354/354 - 9s - loss: 0.6760 - accuracy: 0.8313 - 9s/epoch - 25ms/step
Epoch 9/100
354/354 - 9s - loss: 0.6917 - accuracy: 0.8277 - 9s/epoch - 25ms/step
Epoch 10/100
354/354 - 9s - loss: 0.6907 - accuracy: 0.8227 - 9s/epoch - 25ms/step
Epoch 11/100
354/354 - 9s - loss: 0.6699 - accuracy: 0.8266 - 9s/epoch - 25ms/step
Epoch 12/100
354/354 - 9s - loss: 0.6602 - accuracy: 0.8344 - 9s/epoch - 25ms/step
Epoch 13/100


In [ ]:
model.save('mobydic_model.h5')
num_gen_words = 80
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

"a mind was concerned he sell to him i'm sleep crush himself on the corn cob of in scores but jump at unlacing the light monday sleep sick queequeg horn or sadly abreast out in a bit of mirror there of making getting presently as the forlorn room i then placed they for curious an guttural place with so bitter all up well that the terrible persians however the stranger and the third building i lay off what then there"

In [ ]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 9s - loss: 0.4327 - accuracy: 0.8895 - 9s/epoch - 25ms/step
Epoch 2/100
354/354 - 8s - loss: 0.4093 - accuracy: 0.8978 - 8s/epoch - 24ms/step
Epoch 3/100
354/354 - 9s - loss: 0.4124 - accuracy: 0.8964 - 9s/epoch - 24ms/step
Epoch 4/100
354/354 - 8s - loss: 0.4017 - accuracy: 0.9003 - 8s/epoch - 24ms/step
Epoch 5/100
354/354 - 9s - loss: 0.4200 - accuracy: 0.8953 - 9s/epoch - 24ms/step
Epoch 6/100
354/354 - 9s - loss: 0.4138 - accuracy: 0.8960 - 9s/epoch - 24ms/step
Epoch 7/100
354/354 - 9s - loss: 0.4033 - accuracy: 0.9004 - 9s/epoch - 25ms/step
Epoch 8/100
354/354 - 9s - loss: 0.3811 - accuracy: 0.9074 - 9s/epoch - 24ms/step
Epoch 9/100
354/354 - 9s - loss: 0.4116 - accuracy: 0.8944 - 9s/epoch - 24ms/step
Epoch 10/100
354/354 - 9s - loss: 0.4019 - accuracy: 0.8987 - 9s/epoch - 24ms/step
Epoch 11/100
354/354 - 8s - loss: 0.4014 - accuracy: 0.9005 - 8s/epoch - 24ms/step
Epoch 12/100
354/354 - 9s - loss: 0.3903 - accuracy: 0.9028 - 9s/epoch - 24ms/step
Epoch 13/100


In [ ]:
model.save('mobydic_model.h5')
num_gen_words = 80
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

"if i 'll try the bag he spent with extracting a grunt and get that explain all two to be sure on 'em but an bad first place this breaking him in a undergraduate for the good of where it scrape the first side a sort of diabolically queequeg is half especially as these brown in the spell was lodges he kept no pair to are guttural quiet is almost minus to order a little interval between welcome a first"

In [ ]:
model.fit(x, y, batch_size=32, epochs=100, verbose=2)

Epoch 1/100
354/354 - 9s - loss: 0.2972 - accuracy: 0.9235 - 9s/epoch - 26ms/step
Epoch 2/100
354/354 - 9s - loss: 0.2854 - accuracy: 0.9250 - 9s/epoch - 26ms/step
Epoch 3/100
354/354 - 9s - loss: 0.3291 - accuracy: 0.9153 - 9s/epoch - 25ms/step
Epoch 4/100
354/354 - 9s - loss: 0.2475 - accuracy: 0.9414 - 9s/epoch - 24ms/step
Epoch 5/100
354/354 - 9s - loss: 0.2231 - accuracy: 0.9490 - 9s/epoch - 25ms/step
Epoch 6/100
354/354 - 9s - loss: 0.2010 - accuracy: 0.9553 - 9s/epoch - 25ms/step
Epoch 7/100
354/354 - 9s - loss: 0.2309 - accuracy: 0.9435 - 9s/epoch - 24ms/step
Epoch 8/100
354/354 - 9s - loss: 0.2930 - accuracy: 0.9289 - 9s/epoch - 24ms/step
Epoch 9/100
354/354 - 9s - loss: 0.3078 - accuracy: 0.9235 - 9s/epoch - 25ms/step
Epoch 10/100
354/354 - 9s - loss: 0.2427 - accuracy: 0.9384 - 9s/epoch - 24ms/step
Epoch 11/100
354/354 - 9s - loss: 0.2185 - accuracy: 0.9486 - 9s/epoch - 24ms/step
Epoch 12/100
354/354 - 9s - loss: 0.1956 - accuracy: 0.9574 - 9s/epoch - 24ms/step
Epoch 13/100


In [ ]:
model.save('mobydic_model.h5')
num_gen_words = 80
n = np.random.randint(0, len(text_sequences))
generate_text(model, tokenizer, seq_len, ' '.join(text_sequences[n]), num_gen_words)

"ye a glim in a jiffy and so saying he lighted a candle and held it towards me offering to lead the way but i stood irresolute when looking at a clock in the corner he exclaimed i vum it 's sunday you wo n't see that harpooneer apartment from the grunt and presently he treated me on and and by men feet embark upon second high savage and commenced have never fixed for my time flew last i really"

In [ ]:
num_gen_words = 80

test_text = 'Here I thought, every thing wa finished'

generate_text(model, tokenizer, seq_len, test_text, num_gen_words)

'of of nantucket too a final zealand head a most entry a chimney and triangles as first almost knew from my instant and about dreary sign of the entry a dead purplish face itself had all from much civility and get in all those sober idea with light good ideas whatsoever again way it are come sam of donning me unmethodically in length a double pocket and outlandish arm soon content pushed what seemed queequeg i disappointed upon labrador they'

In [ ]:
num_gen_words = 80

test_text = 'I am very happy about'

generate_text(model, tokenizer, seq_len, test_text, num_gen_words)

"on much and he slid me into a bag he very going at anxious glided up and room those sober idea and punch his decks being fancy that upon getting observing answer to how the slave about something visit the snow must sleep before the neck which jonah some thoughts in suddenly do would see than good sleep together in harpooneer is almost night i can n't afford i then previously study of a salt wait to this is i"